<a href="https://colab.research.google.com/github/AstroBesat-SoftW/AstroBesat-SoftW/blob/main/5_versiyon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
# DOSYA ADI: teknofest_gercek_simulasyon.py
import numpy as np
import pandas as pd
import pickle
import joblib
import xgboost as xgb
import lightgbm as lgb
import os

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import Callback, EarlyStopping
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, cohen_kappa_score

# --- 1. ADIM: VERİ ÜRETİMİ (ŞARTNAMEYE BİREBİR UYGUN) ---
print(">> SİMÜLASYON BAŞLIYOR...")
print(">> Hedef: 4000 Eğitim Verisi + 2500 Test Verisi (External Validation)")

def veri_uret(n, tip="Eğitim"):
    print(f"   + {n} adet '{tip}' verisi üretiliyor...")
    dna_seqs, prot_seqs, bio_feats, labels = [], [], [], []
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")

    for _ in range(n):
        # 11 harfli DNA/Protein (Yarışma örneği)
        dna = ''.join(np.random.choice(list('ACGT'), size=11))
        prot = ''.join(np.random.choice(amino_acids, size=11))

        # Sayısal Özellikler
        risk = np.random.beta(2, 2)
        maf = np.random.exponential(0.05)
        cons = np.random.uniform(0, 10)
        hydro = np.random.uniform(-5, 5)
        polar = np.random.uniform(-3, 3)
        weight = np.random.uniform(-50, 50)

        # Etiketleme (Patojenik/Benign)
        score = (risk * 0.4) + (cons/10 * 0.2) + ((0.5 - maf)*2 * 0.2) + (abs(hydro)/5 * 0.2)
        score += np.random.normal(0, 0.05)
        label = 1 if score > 0.65 else 0

        dna_seqs.append(dna)
        prot_seqs.append(prot)
        bio_feats.append([risk, maf, cons, hydro, polar, weight])
        labels.append(label)

    df_feat = pd.DataFrame(bio_feats, columns=['Risk', 'MAF', 'Cons', 'Hydro', 'Polar', 'Weight'])
    return dna_seqs, prot_seqs, df_feat.values.astype('float32'), np.array(labels)

# 1.1. EĞİTİM SETİ (Bize verilen etiketli veri) -> 4000 Adet
dna_train, prot_train, num_train, y_train = veri_uret(4000, "EĞİTİM SETİ")

# 1.2. TEST SETİ (Yarışma anında verilecek kapalı veri) -> 2500 Adet
dna_test_ext, prot_test_ext, num_test_ext, y_test_ext = veri_uret(2500, "DIŞ TEST SETİ")


# --- 2. ADIM: VERİ HAZIRLIĞI ---
# Tokenizer'ı sadece EĞİTİM setine göre kuruyoruz (Test setini görmemiş gibi davranmalı)
dna_tok = Tokenizer(char_level=True)
dna_tok.fit_on_texts(dna_train)

prot_tok = Tokenizer(char_level=True)
prot_tok.fit_on_texts(prot_train)

# Sequence Dönüşümleri
# Eğitim Seti
X_dna_tr_all = pad_sequences(dna_tok.texts_to_sequences(dna_train), maxlen=11, padding='post')
X_prot_tr_all = pad_sequences(prot_tok.texts_to_sequences(prot_train), maxlen=11, padding='post')

# External Test Seti
X_dna_ext = pad_sequences(dna_tok.texts_to_sequences(dna_test_ext), maxlen=11, padding='post')
X_prot_ext = pad_sequences(prot_tok.texts_to_sequences(prot_test_ext), maxlen=11, padding='post')

# --- 3. ADIM: MODEL İÇİ BÖLÜMLEME (STACKING STRATEJİSİ) ---
# 4000 verinin hepsini modele gömemeyiz, Stacking için "görülmemiş" veriye ihtiyaç var.
# %75 Base Model Eğitimi (3000 Veri)
# %25 Meta Model Eğitimi (1000 Veri) - Buna "Validation" da diyebiliriz.

X_dna_base, X_dna_meta, X_prot_base, X_prot_meta, X_num_base, X_num_meta, y_base, y_meta = train_test_split(
    X_dna_tr_all, X_prot_tr_all, num_train, y_train,
    test_size=0.25, random_state=42
)

print(f"\n>> VERİ DAĞILIMI:")
print(f"   - Base Model Eğitimi (CNN/XGB) : {len(y_base)} veri")
print(f"   - Meta Model Eğitimi (Logistic): {len(y_meta)} veri")
print(f"   - Final Test (Yarışma Verisi)  : {len(y_test_ext)} veri")


# --- 4. ADIM: F1 TAKİPÇİSİ (CALLBACK) ---
class TeknofestBestEpoch(Callback):
    def __init__(self, val_data, model_name):
        super().__init__()
        self.val_data = val_data
        self.best_f1 = 0.0
        self.model_name = model_name

    def on_epoch_end(self, epoch, logs={}):
        # Tahmin yap
        val_probs = self.model.predict(self.val_data[0], verbose=0)
        val_preds = (val_probs > 0.5).astype(int)

        # F1 Hesapla
        _f1 = f1_score(self.val_data[1], val_preds)

        # İyileşme varsa kaydet
        if _f1 > self.best_f1:
            print(f"   ★ Epoch {epoch+1}: F1 {_f1:.4f} -> {self.model_name} Kaydediliyor...")
            self.best_f1 = _f1
            self.model.save(f'model_stack/{self.model_name}.h5')
        else:
            print(f"     Epoch {epoch+1}: F1 {_f1:.4f} (En iyi: {self.best_f1:.4f})")

# Klasör kontrolü
if not os.path.exists('model_stack'): os.makedirs('model_stack')


# --- 5. ADIM: MODELLERİN EĞİTİMİ ---

# === [1] CNN MODELİ ===
print("\n>> [1/4] CNN Modeli Eğitiliyor (Deep Learning)...")

in_dna = Input(shape=(11,))
emb_dna = Embedding(len(dna_tok.word_index)+1, 8)(in_dna)
x1 = GlobalMaxPooling1D()(Conv1D(32, 3, activation='relu')(emb_dna))

in_prot = Input(shape=(11,))
emb_prot = Embedding(len(prot_tok.word_index)+1, 8)(in_prot)
x2 = GlobalMaxPooling1D()(Conv1D(32, 3, activation='relu')(emb_prot))

in_num_layer = Input(shape=(6,))
x3 = BatchNormalization()(Dense(32, activation='relu')(in_num_layer))

merged = Concatenate()([x1, x2, x3])
z = Dropout(0.5)(Dense(64, activation='relu')(merged))
out = Dense(1, activation='sigmoid')(z)

model_cnn = Model(inputs=[in_dna, in_prot, in_num_layer], outputs=out)
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callback'ler
cnn_callback = TeknofestBestEpoch(
    val_data=([X_dna_meta, X_prot_meta, X_num_meta], y_meta),
    model_name="best_cnn"
)
early_stop = EarlyStopping(monitor='val_loss', patience=15, verbose=1)

model_cnn.fit(
    [X_dna_base, X_prot_base, X_num_base], y_base,
    validation_data=([X_dna_meta, X_prot_meta, X_num_meta], y_meta),
    epochs=100,
    batch_size=32,
    verbose=0, # Detayları kapattık, özel callback yazacak
    callbacks=[cnn_callback, early_stop]
)

# En iyi CNN'i geri yükle (Stacking için)
print("   >> En iyi CNN ağırlıkları yükleniyor...")
model_cnn = load_model('model_stack/best_cnn.h5')


# === [2] XGBOOST MODELİ ===
print("\n>> [2/4] XGBoost Modeli Eğitiliyor...")
model_xgb = xgb.XGBClassifier(n_estimators=150, learning_rate=0.05, max_depth=4, eval_metric='logloss', use_label_encoder=False)
model_xgb.fit(X_num_base, y_base)
print("   >> XGBoost Hazır.")


# === [3] LIGHTGBM MODELİ ===
print("\n>> [3/4] LightGBM Modeli Eğitiliyor...")
model_lgbm = lgb.LGBMClassifier(n_estimators=150, learning_rate=0.05, num_leaves=20, verbose=-1)
model_lgbm.fit(X_num_base, y_base)
print("   >> LightGBM Hazır.")


# === [4] STACKING (META-LEARNER) ===
print("\n>> [4/4] Modellerin Güçleri Birleştiriliyor (Stacking)...")

# Meta seti üzerinde tahminler (Validation verisi gibi düşün)
p_cnn = model_cnn.predict([X_dna_meta, X_prot_meta, X_num_meta], verbose=0).flatten()
p_xgb = model_xgb.predict_proba(X_num_meta)[:, 1]
p_lgbm = model_lgbm.predict_proba(X_num_meta)[:, 1]

# Bu tahminleri girdi olarak kullan
X_stack_meta = np.column_stack((p_cnn, p_xgb, p_lgbm))

# Patron (Meta) Modeli Eğit
meta_model = LogisticRegression()
meta_model.fit(X_stack_meta, y_meta)

print("   >> Stacking Modeli Hazır. Ağırlıklar:")
print(f"      CNN: {meta_model.coef_[0][0]:.2f}, XGB: {meta_model.coef_[0][1]:.2f}, LGBM: {meta_model.coef_[0][2]:.2f}")


# --- 6. ADIM: FİNAL SINAVI (EXTERNAL VALIDATION) ---
print("\n" + "="*60)
print("🧪 FINAL EXTERNAL VALIDATION (2500 Verilik Test)")
print("="*60)

# Dış Test Seti Üzerinde Tahminler
t_cnn = model_cnn.predict([X_dna_ext, X_prot_ext, num_test_ext], verbose=0).flatten()
t_xgb = model_xgb.predict_proba(num_test_ext)[:, 1]
t_lgbm = model_lgbm.predict_proba(num_test_ext)[:, 1]

# Stacking Tahmini
X_stack_ext = np.column_stack((t_cnn, t_xgb, t_lgbm))
final_preds = meta_model.predict(X_stack_ext)

# Metrikler
f1 = f1_score(y_test_ext, final_preds)
acc = accuracy_score(y_test_ext, final_preds)
prec = precision_score(y_test_ext, final_preds)
rec = recall_score(y_test_ext, final_preds)
kappa = cohen_kappa_score(y_test_ext, final_preds)

print(f"| METRİK               | DEĞER     | AÇIKLAMA")
print(f"|----------------------|-----------|----------------------------------")
print(f"| F1 Score (ÖNEMLİ)    | %{f1*100:.2f}    | Yarışma sıralama kriteri")
print(f"| Accuracy (Doğruluk)  | %{acc*100:.2f}    | Toplam doğru bilme oranı")
print(f"| Recall (Duyarlılık)  | %{rec*100:.2f}    | Hastaların kaçı yakalandı?")
print(f"| Precision (Kesinlik) | %{prec*100:.2f}    | Hasta dediklerinin kaçı hasta?")
print(f"| Kappa Score          | {kappa:.4f}    | { 'Mükemmel' if kappa>0.8 else 'İyi' if kappa>0.6 else 'Orta'} Uyum")
print("="*60)

# Kaydet
joblib.dump(meta_model, 'model_stack/meta_model_final.pkl')
joblib.dump(model_xgb, 'model_stack/xgb_final.pkl')
joblib.dump(model_lgbm, 'model_stack/lgbm_final.pkl')
with open('model_stack/dna_tok_final.pickle', 'wb') as f: pickle.dump(dna_tok, f)
with open('model_stack/prot_tok_final.pickle', 'wb') as f: pickle.dump(prot_tok, f)

print("\n✅ SİMÜLASYON TAMAMLANDI. Dosyalar 'model_stack' klasöründe.")

>> SİMÜLASYON BAŞLIYOR...
>> Hedef: 4000 Eğitim Verisi + 2500 Test Verisi (External Validation)
   + 4000 adet 'EĞİTİM SETİ' verisi üretiliyor...
   + 2500 adet 'DIŞ TEST SETİ' verisi üretiliyor...

>> VERİ DAĞILIMI:
   - Base Model Eğitimi (CNN/XGB) : 3000 veri
   - Meta Model Eğitimi (Logistic): 1000 veri
   - Final Test (Yarışma Verisi)  : 2500 veri

>> [1/4] CNN Modeli Eğitiliyor (Deep Learning)...


   ★ Epoch 1: F1 0.4905 -> best_cnn Kaydediliyor...
     Epoch 2: F1 0.4635 (En iyi: 0.4905)
     Epoch 3: F1 0.3632 (En iyi: 0.4905)
     Epoch 4: F1 0.1479 (En iyi: 0.4905)
     Epoch 5: F1 0.0209 (En iyi: 0.4905)


   ★ Epoch 6: F1 0.5103 -> best_cnn Kaydediliyor...


   ★ Epoch 7: F1 0.5255 -> best_cnn Kaydediliyor...


   ★ Epoch 8: F1 0.7836 -> best_cnn Kaydediliyor...
     Epoch 9: F1 0.7651 (En iyi: 0.7836)


...

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [01:22:20] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


   >> XGBoost Hazır.

>> [3/4] LightGBM Modeli Eğitiliyor...
   >> LightGBM Hazır.

>> [4/4] Modellerin Güçleri Birleştiriliyor (Stacking)...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


   >> Stacking Modeli Hazır. Ağırlıklar:
      CNN: 3.65, XGB: 2.00, LGBM: 1.28

🧪 FINAL EXTERNAL VALIDATION (2500 Verilik Test)
| METRİK               | DEĞER     | AÇIKLAMA
|----------------------|-----------|----------------------------------
| F1 Score (ÖNEMLİ)    | %82.35    | Yarışma sıralama kriteri
| Accuracy (Doğruluk)  | %89.56    | Toplam doğru bilme oranı
| Recall (Duyarlılık)  | %80.03    | Hastaların kaçı yakalandı?
| Precision (Kesinlik) | %84.82    | Hasta dediklerinin kaçı hasta?
| Kappa Score          | 0.7495    | İyi Uyum

✅ SİMÜLASYON TAMAMLANDI. Dosyalar 'model_stack' klasöründe.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [84]:
# DOSYA ADI: test_stacking_final.py
import pandas as pd
import numpy as np
import pickle
import joblib
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, cohen_kappa_score

print(">> 1. SİSTEM BAŞLATILIYOR...")

# --- A. MODEL VE SÖZLÜKLERİ YÜKLE (Yeni dosya isimlerine göre) ---
print(">> Modeller ve Sözlükler yükleniyor...")
try:
    # Tokenizerlar
    with open('model_stack/dna_tok_final.pickle', 'rb') as f: dna_tok = pickle.load(f)
    with open('model_stack/prot_tok_final.pickle', 'rb') as f: prot_tok = pickle.load(f)

    # Modeller (Eğitim kodunda kaydettiğimiz isimler)
    model_cnn = load_model('model_stack/best_cnn.h5')
    model_xgb = joblib.load('model_stack/xgb_final.pkl')
    model_lgbm = joblib.load('model_stack/lgbm_final.pkl')
    meta_model = joblib.load('model_stack/meta_model_final.pkl')
    print("✅ Tüm beyinler (CNN, XGB, LGBM, Meta) başarıyla yüklendi.")
except FileNotFoundError:
    print("❌ HATA: Model dosyaları bulunamadı. Lütfen önce eğitim kodunu çalıştırın.")
    exit()

# --- B. 2.500 ADET TEST VERİSİ ÜRET (EXTERNAL VALIDATION SİMÜLASYONU) ---
print(f"\n>> 2. Yarışma Senaryosu: 2.500 Adet YENİ Test Verisi Geliyor...")

def veri_uret_test(n):
    dna_seqs, prot_seqs, bio_feats, labels = [], [], [], []
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")

    for _ in range(n):
        # Rastgele DNA/Protein
        dna = ''.join(np.random.choice(list('ACGT'), size=11))
        prot = ''.join(np.random.choice(amino_acids, size=11))

        # Sayısal Özellikler
        risk = np.random.beta(2, 2)
        maf = np.random.exponential(0.05)
        cons = np.random.uniform(0, 10)
        hydro = np.random.uniform(-5, 5)
        polar = np.random.uniform(-3, 3)
        weight = np.random.uniform(-50, 50)

        # Gerçek Etiket (Simülasyon Kuralı)
        score = (risk * 0.4) + (cons/10 * 0.2) + ((0.5 - maf)*2 * 0.2) + (abs(hydro)/5 * 0.2)
        score += np.random.normal(0, 0.05)
        label = 1 if score > 0.65 else 0

        dna_seqs.append(dna)
        prot_seqs.append(prot)
        bio_feats.append([risk, maf, cons, hydro, polar, weight])
        labels.append(label)

    df_feat = pd.DataFrame(bio_feats, columns=['Risk', 'MAF', 'Cons', 'Hydro', 'Polar', 'Weight'])
    return dna_seqs, prot_seqs, df_feat, np.array(labels)

# Veriyi Canlı Üret
ham_dna, ham_prot, df_num, y_gercek = veri_uret_test(2500)

# --- C. VERİYİ MODELE HAZIRLA ---
print(">> Veriler işleniyor...")

# 1. Tokenize Et (Sequence)
seq_dna = pad_sequences(dna_tok.texts_to_sequences(ham_dna), maxlen=11, padding='post')
seq_prot = pad_sequences(prot_tok.texts_to_sequences(ham_prot), maxlen=11, padding='post')

# 2. Sayısal Veriyi float32 yap (Hata önleyici)
X_num = df_num.values.astype('float32')

# --- D. TAHMİN (STACKING) ---
print(">> Modeller tahmin yürütüyor...")

# 1. Aşama: Uzman Modellerin Tahminleri
p_cnn = model_cnn.predict([seq_dna, seq_prot, X_num], verbose=0).flatten()
p_xgb = model_xgb.predict_proba(X_num)[:, 1]
p_lgbm = model_lgbm.predict_proba(X_num)[:, 1]

# 2. Aşama: Meta Modelin Kararı
stack_matrix = np.column_stack((p_cnn, p_xgb, p_lgbm))
final_preds = meta_model.predict(stack_matrix)

# --- E. RAPORLAMA ---
print("\n" + "="*50)
print("📊 2.500 VERİLİK FİNAL TEST SONUCU")
print("="*50)

acc = accuracy_score(y_gercek, final_preds)
f1 = f1_score(y_gercek, final_preds)
prec = precision_score(y_gercek, final_preds)
rec = recall_score(y_gercek, final_preds)
kappa = cohen_kappa_score(y_gercek, final_preds)

print(f"✅ Accuracy  : %{acc*100:.2f}")
print(f"⚖️ F1 Score  : %{f1*100:.2f} (Yarışma Sıralama Kriteri)")
print(f"🎯 Precision : %{prec*100:.2f}")
print(f"🔍 Recall    : %{rec*100:.2f}")
print(f"🎲 Kappa     : {kappa:.4f} ", end="")

if kappa > 0.8: print("(Mükemmel 🌟)")
elif kappa > 0.6: print("(İyi 👍)")
elif kappa > 0.4: print("(Orta 😐)")
else: print("(Kötü ⚠️)")
print("="*50)

# --- F. TEKİL ÖRNEK (GÖZLE KONTROL) ---
print("\n>> Örnek bir hasta dosyası inceleniyor...")
idx = np.random.randint(0, 2500) # Rastgele bir kişi seç

print("-" * 40)
print(f"Hasta ID       : {idx}")
print(f"DNA Dizisi     : {ham_dna[idx]}")
print(f"Protein        : {ham_prot[idx]}")
print(f"Risk Skoru     : {df_num.iloc[idx]['Risk']:.4f}")
print("-" * 40)
print(f"🔍 CNN Olasılık : %{p_cnn[idx]*100:.1f}")
print(f"🌲 XGB Olasılık : %{p_xgb[idx]*100:.1f}")
print(f"🍃 LGBM Olasılık: %{p_lgbm[idx]*100:.1f}")
print("-" * 40)
print(f"🤖 SİSTEM KARARI: {'HASTA ⚠️' if final_preds[idx]==1 else 'SAĞLIKLI ✅'}")
print(f"⚕️ GERÇEK DURUM : {'HASTA ⚠️' if y_gercek[idx]==1 else 'SAĞLIKLI ✅'}")

if final_preds[idx] == y_gercek[idx]:
    print("SONUÇ: ✅ DOĞRU TANI")
else:
    print("SONUÇ: ❌ YANLIŞ TANI")
print("="*50)

>> 1. SİSTEM BAŞLATILIYOR...
>> Modeller ve Sözlükler yükleniyor...
✅ Tüm beyinler (CNN, XGB, LGBM, Meta) başarıyla yüklendi.

>> 2. Yarışma Senaryosu: 2.500 Adet YENİ Test Verisi Geliyor...
>> Veriler işleniyor...
>> Modeller tahmin yürütüyor...

📊 2.500 VERİLİK FİNAL TEST SONUCU
✅ Accuracy  : %88.76
⚖️ F1 Score  : %81.79 (Yarışma Sıralama Kriteri)
🎯 Precision : %84.25
🔍 Recall    : %79.47
🎲 Kappa     : 0.7367 (İyi 👍)

>> Örnek bir hasta dosyası inceleniyor...
----------------------------------------
Hasta ID       : 1625
DNA Dizisi     : AATTATCCTCC
Protein        : DHHLTPQYDGT
Risk Skoru     : 0.7388
----------------------------------------
🔍 CNN Olasılık : %11.3
🌲 XGB Olasılık : %15.3
🍃 LGBM Olasılık: %13.4
----------------------------------------
🤖 SİSTEM KARARI: SAĞLIKLI ✅
⚕️ GERÇEK DURUM : SAĞLIKLI ✅
SONUÇ: ✅ DOĞRU TANI


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
